In [240]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [241]:
game = pd.read_csv('rawg_games_4k.csv')

In [242]:
game

,id,name,slug,released,metacritic,rating,ratings_count,platforms,genres,stores,added,tags
0,3498,Grand Theft Auto V,grand-theft-auto-v,2013-09-17,92.0,4.47,7235,PC|PlayStation 5|Xbox Series S/X|PlayStation 4...,Action,Steam|PlayStation Store|Epic Games|Xbox 360 St...,22280,Singleplayer|Steam Achievements|Multiplayer|Fu...
1,3328,The Witcher 3: Wild Hunt,the-witcher-3-wild-hunt,2015-05-18,92.0,4.64,7052,PlayStation 5|Xbox Series S/X|macOS|PlayStatio...,Action|RPG,GOG|PlayStation Store|Steam|Xbox Store|Nintend...,21954,Singleplayer|Full controller support|Atmospher...
2,4200,Portal 2,portal-2,2011-04-18,95.0,4.58,5991,PlayStation 3|PC|Xbox 360|Linux|macOS|Xbox One,Shooter|Puzzle,Xbox Store|Steam|PlayStation Store|Xbox 360 Store,20716,Singleplayer|Steam Achievements|Multiplayer|Fu...
3,4291,Counter-Strike: Global Offensive,counter-strike-global-offensive,2012-08-21,81.0,3.57,3603,PC|Linux|Xbox 360|PlayStation 3,Shooter,PlayStation Store|Steam|Xbox 360 Store,18318,Steam Achievements|Multiplayer|Full controller...
4,5286,Tomb Raider (2013),tomb-raider,2013-03-05,86.0,4.06,4048,PlayStation 3|Xbox 360|macOS|PC|Xbox One|PlayS...,Action,Xbox 360 Store|Steam|PlayStation Store|Google ...,17697,Singleplayer|Multiplayer|Full controller suppo...
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,17539,Data Hacker: Reboot,data-hacker-reboot,2015-04-17,NaN,1.67,21,PC,Adventure|RPG|Indie,Steam,700,Singleplayer|Steam Achievements|Steam Cloud|st...
3996,13285,A.V.A. Alliance of Valiant Arms,ava-alliance-of-valiant-arms,2011-06-14,NaN,1.88,26,PC,Action,Steam,700,Singleplayer|Steam Achievements|Multiplayer|st...
3997,12530,MIND: Path to Thalamus Enhanced Edition,mind-path-to-thalamus-enhanced-edition,2014-08-05,75.0,3.03,37,Linux|macOS|PC,Adventure|Strategy|Indie,Steam,700,Singleplayer|Steam Achievements|Full controlle...
3998,12116,AdVenture Communist,adventure-communist,2016-08-23,NaN,2.74,53,macOS|PC|Android|iOS,Strategy|Simulation|Casual|Indie,Steam|Google Play|App Store,700,Singleplayer|Great Soundtrack|Horror|Funny|Fre...


In [243]:
game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4000 non-null   int64  
 1   name           4000 non-null   object 
 2   slug           4000 non-null   object 
 3   released       3977 non-null   object 
 4   metacritic     2571 non-null   float64
 5   rating         4000 non-null   float64
 6   ratings_count  4000 non-null   int64  
 7   platforms      4000 non-null   object 
 8   genres         3979 non-null   object 
 9   stores         3943 non-null   object 
 10  added          4000 non-null   int64  
 11  tags           3959 non-null   object 
dtypes: float64(2), int64(3), object(7)
memory usage: 375.1+ KB


In [244]:
game.describe()

,id,metacritic,rating,ratings_count,added
count,4000.000000,2571.000000,4000.000000,4000.000000,4000.000000
mean,83849.048750,76.574096,3.369525,338.064750,2277.414000
std,165597.274684,9.484385,0.724806,551.514584,2294.392945
min,21.000000,23.000000,0.000000,2.000000,699.000000
25%,9453.000000,71.000000,2.910000,63.000000,945.750000
50%,17123.500000,78.000000,3.480000,142.000000,1411.000000
75%,46405.500000,83.000000,3.920000,364.250000,2527.250000
max,993875.000000,99.000000,4.810000,7235.000000,22280.000000


In [245]:
game['name'].duplicated()

,name
0,False
1,False
2,False
3,False
4,False
...,...
3995,False
3996,False
3997,False
3998,False


In [246]:
game = game[['id', 'name', 'platforms', 'genres', 'stores', 'tags']]

In [247]:
game

,id,name,platforms,genres,stores,tags
0,3498,Grand Theft Auto V,PC|PlayStation 5|Xbox Series S/X|PlayStation 4...,Action,Steam|PlayStation Store|Epic Games|Xbox 360 St...,Singleplayer|Steam Achievements|Multiplayer|Fu...
1,3328,The Witcher 3: Wild Hunt,PlayStation 5|Xbox Series S/X|macOS|PlayStatio...,Action|RPG,GOG|PlayStation Store|Steam|Xbox Store|Nintend...,Singleplayer|Full controller support|Atmospher...
2,4200,Portal 2,PlayStation 3|PC|Xbox 360|Linux|macOS|Xbox One,Shooter|Puzzle,Xbox Store|Steam|PlayStation Store|Xbox 360 Store,Singleplayer|Steam Achievements|Multiplayer|Fu...
3,4291,Counter-Strike: Global Offensive,PC|Linux|Xbox 360|PlayStation 3,Shooter,PlayStation Store|Steam|Xbox 360 Store,Steam Achievements|Multiplayer|Full controller...
4,5286,Tomb Raider (2013),PlayStation 3|Xbox 360|macOS|PC|Xbox One|PlayS...,Action,Xbox 360 Store|Steam|PlayStation Store|Google ...,Singleplayer|Multiplayer|Full controller suppo...
...,...,...,...,...,...,...
3995,17539,Data Hacker: Reboot,PC,Adventure|RPG|Indie,Steam,Singleplayer|Steam Achievements|Steam Cloud|st...
3996,13285,A.V.A. Alliance of Valiant Arms,PC,Action,Steam,Singleplayer|Steam Achievements|Multiplayer|st...
3997,12530,MIND: Path to Thalamus Enhanced Edition,Linux|macOS|PC,Adventure|Strategy|Indie,Steam,Singleplayer|Steam Achievements|Full controlle...
3998,12116,AdVenture Communist,macOS|PC|Android|iOS,Strategy|Simulation|Casual|Indie,Steam|Google Play|App Store,Singleplayer|Great Soundtrack|Horror|Funny|Fre...


In [248]:
game['platforms'].isnull().sum()

np.int64(0)

In [249]:
game['genres'].isnull().sum()

np.int64(21)

In [250]:
game = game.dropna(subset=['genres', 'stores', 'tags'])

In [251]:
game

,id,name,platforms,genres,stores,tags
0,3498,Grand Theft Auto V,PC|PlayStation 5|Xbox Series S/X|PlayStation 4...,Action,Steam|PlayStation Store|Epic Games|Xbox 360 St...,Singleplayer|Steam Achievements|Multiplayer|Fu...
1,3328,The Witcher 3: Wild Hunt,PlayStation 5|Xbox Series S/X|macOS|PlayStatio...,Action|RPG,GOG|PlayStation Store|Steam|Xbox Store|Nintend...,Singleplayer|Full controller support|Atmospher...
2,4200,Portal 2,PlayStation 3|PC|Xbox 360|Linux|macOS|Xbox One,Shooter|Puzzle,Xbox Store|Steam|PlayStation Store|Xbox 360 Store,Singleplayer|Steam Achievements|Multiplayer|Fu...
3,4291,Counter-Strike: Global Offensive,PC|Linux|Xbox 360|PlayStation 3,Shooter,PlayStation Store|Steam|Xbox 360 Store,Steam Achievements|Multiplayer|Full controller...
4,5286,Tomb Raider (2013),PlayStation 3|Xbox 360|macOS|PC|Xbox One|PlayS...,Action,Xbox 360 Store|Steam|PlayStation Store|Google ...,Singleplayer|Multiplayer|Full controller suppo...
...,...,...,...,...,...,...
3995,17539,Data Hacker: Reboot,PC,Adventure|RPG|Indie,Steam,Singleplayer|Steam Achievements|Steam Cloud|st...
3996,13285,A.V.A. Alliance of Valiant Arms,PC,Action,Steam,Singleplayer|Steam Achievements|Multiplayer|st...
3997,12530,MIND: Path to Thalamus Enhanced Edition,Linux|macOS|PC,Adventure|Strategy|Indie,Steam,Singleplayer|Steam Achievements|Full controlle...
3998,12116,AdVenture Communist,macOS|PC|Android|iOS,Strategy|Simulation|Casual|Indie,Steam|Google Play|App Store,Singleplayer|Great Soundtrack|Horror|Funny|Fre...


In [252]:
def blank_space_remover(obj):
  return obj.str.replace(' ', '', regex=False)

In [253]:
game[['platforms', 'genres', 'stores', 'tags']] = game[['platforms', 'genres', 'stores', 'tags']].apply(blank_space_remover)

/tmp/ipython-input-1393245905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game[['platforms', 'genres', 'stores', 'tags']] = game[['platforms', 'genres', 'stores', 'tags']].apply(blank_space_remover)


In [254]:
def pipe_remover(obj):
  return obj.str.replace('|', ' ', regex=False)

In [255]:
game[['platforms', 'genres', 'stores', 'tags']] = game[['platforms', 'genres', 'stores', 'tags']].apply(pipe_remover)

/tmp/ipython-input-1467880456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game[['platforms', 'genres', 'stores', 'tags']] = game[['platforms', 'genres', 'stores', 'tags']].apply(pipe_remover)


In [256]:
game

,id,name,platforms,genres,stores,tags
0,3498,Grand Theft Auto V,PC PlayStation5 XboxSeriesS/X PlayStation4 Pla...,Action,Steam PlayStationStore EpicGames Xbox360Store ...,Singleplayer SteamAchievements Multiplayer Ful...
1,3328,The Witcher 3: Wild Hunt,PlayStation5 XboxSeriesS/X macOS PlayStation4 ...,Action RPG,GOG PlayStationStore Steam XboxStore NintendoS...,Singleplayer Fullcontrollersupport Atmospheric...
2,4200,Portal 2,PlayStation3 PC Xbox360 Linux macOS XboxOne,Shooter Puzzle,XboxStore Steam PlayStationStore Xbox360Store,Singleplayer SteamAchievements Multiplayer Ful...
3,4291,Counter-Strike: Global Offensive,PC Linux Xbox360 PlayStation3,Shooter,PlayStationStore Steam Xbox360Store,SteamAchievements Multiplayer Fullcontrollersu...
4,5286,Tomb Raider (2013),PlayStation3 Xbox360 macOS PC XboxOne PlayStat...,Action,Xbox360Store Steam PlayStationStore GooglePlay...,Singleplayer Multiplayer Fullcontrollersupport...
...,...,...,...,...,...,...
3995,17539,Data Hacker: Reboot,PC,Adventure RPG Indie,Steam,Singleplayer SteamAchievements SteamCloud stea...
3996,13285,A.V.A. Alliance of Valiant Arms,PC,Action,Steam,Singleplayer SteamAchievements Multiplayer ste...
3997,12530,MIND: Path to Thalamus Enhanced Edition,Linux macOS PC,Adventure Strategy Indie,Steam,Singleplayer SteamAchievements Fullcontrollers...
3998,12116,AdVenture Communist,macOS PC Android iOS,Strategy Simulation Casual Indie,Steam GooglePlay AppStore,Singleplayer GreatSoundtrack Horror Funny Free...


In [257]:
game['tags'] = game['platforms'] + ' ' + game['genres'] + ' ' + game['stores'] + ' ' + game['tags']

/tmp/ipython-input-937292035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game['tags'] = game['platforms'] + ' ' + game['genres'] + ' ' + game['stores'] + ' ' + game['tags']


In [258]:
game

,id,name,platforms,genres,stores,tags
0,3498,Grand Theft Auto V,PC PlayStation5 XboxSeriesS/X PlayStation4 Pla...,Action,Steam PlayStationStore EpicGames Xbox360Store ...,PC PlayStation5 XboxSeriesS/X PlayStation4 Pla...
1,3328,The Witcher 3: Wild Hunt,PlayStation5 XboxSeriesS/X macOS PlayStation4 ...,Action RPG,GOG PlayStationStore Steam XboxStore NintendoS...,PlayStation5 XboxSeriesS/X macOS PlayStation4 ...
2,4200,Portal 2,PlayStation3 PC Xbox360 Linux macOS XboxOne,Shooter Puzzle,XboxStore Steam PlayStationStore Xbox360Store,PlayStation3 PC Xbox360 Linux macOS XboxOne Sh...
3,4291,Counter-Strike: Global Offensive,PC Linux Xbox360 PlayStation3,Shooter,PlayStationStore Steam Xbox360Store,PC Linux Xbox360 PlayStation3 Shooter PlayStat...
4,5286,Tomb Raider (2013),PlayStation3 Xbox360 macOS PC XboxOne PlayStat...,Action,Xbox360Store Steam PlayStationStore GooglePlay...,PlayStation3 Xbox360 macOS PC XboxOne PlayStat...
...,...,...,...,...,...,...
3995,17539,Data Hacker: Reboot,PC,Adventure RPG Indie,Steam,PC Adventure RPG Indie Steam Singleplayer Stea...
3996,13285,A.V.A. Alliance of Valiant Arms,PC,Action,Steam,PC Action Steam Singleplayer SteamAchievements...
3997,12530,MIND: Path to Thalamus Enhanced Edition,Linux macOS PC,Adventure Strategy Indie,Steam,Linux macOS PC Adventure Strategy Indie Steam ...
3998,12116,AdVenture Communist,macOS PC Android iOS,Strategy Simulation Casual Indie,Steam GooglePlay AppStore,macOS PC Android iOS Strategy Simulation Casua...


In [259]:
game = game[['id', 'name', 'tags']]

In [260]:
game

,id,name,tags
0,3498,Grand Theft Auto V,PC PlayStation5 XboxSeriesS/X PlayStation4 Pla...
1,3328,The Witcher 3: Wild Hunt,PlayStation5 XboxSeriesS/X macOS PlayStation4 ...
2,4200,Portal 2,PlayStation3 PC Xbox360 Linux macOS XboxOne Sh...
3,4291,Counter-Strike: Global Offensive,PC Linux Xbox360 PlayStation3 Shooter PlayStat...
4,5286,Tomb Raider (2013),PlayStation3 Xbox360 macOS PC XboxOne PlayStat...
...,...,...,...
3995,17539,Data Hacker: Reboot,PC Adventure RPG Indie Steam Singleplayer Stea...
3996,13285,A.V.A. Alliance of Valiant Arms,PC Action Steam Singleplayer SteamAchievements...
3997,12530,MIND: Path to Thalamus Enhanced Edition,Linux macOS PC Adventure Strategy Indie Steam ...
3998,12116,AdVenture Communist,macOS PC Android iOS Strategy Simulation Casua...


In [261]:
game['tags'][0]

'PC PlayStation5 XboxSeriesS/X PlayStation4 PlayStation3 Xbox360 XboxOne Action Steam PlayStationStore EpicGames Xbox360Store XboxStore Singleplayer SteamAchievements Multiplayer Fullcontrollersupport Atmospheric GreatSoundtrack RPG Co-op OpenWorld cooperative First-Person ThirdPerson Funny Sandbox Comedy Third-PersonShooter Moddable Crime vrmod'

In [262]:
game['tags'] = game['tags'].str.lower()

/tmp/ipython-input-765118275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game['tags'] = game['tags'].str.lower()


In [263]:
game

,id,name,tags
0,3498,Grand Theft Auto V,pc playstation5 xboxseriess/x playstation4 pla...
1,3328,The Witcher 3: Wild Hunt,playstation5 xboxseriess/x macos playstation4 ...
2,4200,Portal 2,playstation3 pc xbox360 linux macos xboxone sh...
3,4291,Counter-Strike: Global Offensive,pc linux xbox360 playstation3 shooter playstat...
4,5286,Tomb Raider (2013),playstation3 xbox360 macos pc xboxone playstat...
...,...,...,...
3995,17539,Data Hacker: Reboot,pc adventure rpg indie steam singleplayer stea...
3996,13285,A.V.A. Alliance of Valiant Arms,pc action steam singleplayer steamachievements...
3997,12530,MIND: Path to Thalamus Enhanced Edition,linux macos pc adventure strategy indie steam ...
3998,12116,AdVenture Communist,macos pc android ios strategy simulation casua...


In [274]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(game['tags']).toarray()

In [275]:
game = game.reset_index(drop=True)

In [276]:
similarity = cosine_similarity(vectors)

In [277]:
similarity

array([[1.        , 0.53033009, 0.525     , ..., 0.32475953, 0.19764235,
        0.46875   ],
       [0.53033009, 1.        , 0.25927249, ..., 0.27216553, 0.186339  ,
        0.47140452],
       [0.525     , 0.25927249, 1.        , ..., 0.4330127 , 0.18973666,
        0.375     ],
       ...,
       [0.32475953, 0.27216553, 0.4330127 , ..., 1.        , 0.41079192,
        0.36084392],
       [0.19764235, 0.186339  , 0.18973666, ..., 0.41079192, 1.        ,
        0.11858541],
       [0.46875   , 0.47140452, 0.375     , ..., 0.36084392, 0.11858541,
        1.        ]])

In [278]:
def recommend(games):
  game_index = game[game['name'] == games].index[0]
  distances = similarity[game_index]
  games_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
  for i in games_list:
        print(game.iloc[i[0]]['name'])

In [279]:
recommend('Data Hacker: Reboot')

Memories of a Vagabond
Space Pilgrim Episode II: Epsilon Indi
Atonement: Scourge of Time
DarkEnd
Unforgiving Trials: The Darkest Crusade
